# Tuning Neural Networks with Normalization - Lab

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will:

- Fit a neural network to normalized data
- Implement and observe the impact of various initialization techniques
- Implement and observe the impact of various optimization techniques

## Load the data

First, run the following cell to import all the neccessary libraries and classes you will need in this lab.

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository):

- The data was split into the training, validate, and test sets
- All the missing values in numeric columns were replaced by the median of those columns
- All the missing values in catetgorical columns were replaced with the word 'missing'
- All the categorical columns were one-hot encoded

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('https://raw.githubusercontent.com/leksea/dsc-tuning-neural-networks-with-normalization-lab-v2-1/master/data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('https://raw.githubusercontent.com/leksea/dsc-tuning-neural-networks-with-normalization-lab-v2-1/master/data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('https://raw.githubusercontent.com/leksea/dsc-tuning-neural-networks-with-normalization-lab-v2-1/master/data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('https://raw.githubusercontent.com/leksea/dsc-tuning-neural-networks-with-normalization-lab-v2-1/master/data/X_train_cat.csv')
X_test_cat = pd.read_csv('https://raw.githubusercontent.com/leksea/dsc-tuning-neural-networks-with-normalization-lab-v2-1/master/data/X_test_cat.csv')
X_val_cat = pd.read_csv('https://raw.githubusercontent.com/leksea/dsc-tuning-neural-networks-with-normalization-lab-v2-1/master/data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('https://raw.githubusercontent.com/leksea/dsc-tuning-neural-networks-with-normalization-lab-v2-1/master/data/y_train.csv')
y_test = pd.read_csv('https://raw.githubusercontent.com/leksea/dsc-tuning-neural-networks-with-normalization-lab-v2-1/master/data/y_test.csv')
y_val = pd.read_csv('https://raw.githubusercontent.com/leksea/dsc-tuning-neural-networks-with-normalization-lab-v2-1/master/data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

As a refresher, preview the training data:

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that!

In the cell below:

- Add an input layer with `n_features` units
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function)
- Add an output layer with 1 unit and `'linear'` activation
- Compile and fit the model

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [6]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=n_features))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer

baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD',
                       loss='mse',
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train,
                   y_train,
                   batch_size=32,
                   epochs=150,
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 1s 24ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [============================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_

## Normalize the Input Data

It's now time to normalize the input data. In the cell below:

- Assign the column names of all numeric columns to `numeric_columns`
- Instantiate a `StandardScaler`
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names)
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well
- Use the provided to combine the scaled numerical and categorical features

In [9]:
# Numeric column names
numeric_columns = X_train_numeric.columns

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = ss_X.fit_transform(X_train_numeric)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=numeric_columns)

# Transform validate and test data
X_val_scaled = ss_X.transform(X_val_numeric)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=numeric_columns)
X_test_scaled =  ss_X.transform(X_test_numeric)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before):

In [10]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD',
                               loss='mse',
                               metrics=['mse'])

In the cell below:
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`)
- Set a batch size of 32 and train for 150 epochs
- Specify the `validation_data` argument as `(X_val, y_val)`
Again, you may get some strange behavior.

In [12]:
# Train the model
normalized_input_model.fit(X_train,
          y_train,
          batch_size=32,
          epochs=150,
          validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 1s 9ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to:

- Fit and transform `y_train`
- Transform `y_val` and `y_test`

In [13]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below:
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`)
- Set a batch size of 32 and train for 150 epochs
- Specify the `validation_data` as `(X_val, y_val_scaled)`

In [14]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD',
                         loss='mse',
                         metrics=['mse'])

# Train the model
normalized_model.fit(X_train,
                     y_train_scaled,
                     batch_size=32,
                     epochs=150,
                     validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 21ms/step - loss: 0.4673 - mse: 0.4673 - val_loss: 0.1818 - val_mse: 0.1818
Epoch 2/150
33/33 [==============================] - 0s 8ms/step - loss: 0.2487 - mse: 0.2487 - val_loss: 0.1552 - val_mse: 0.1552
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2024 - mse: 0.2024 - val_loss: 0.1333 - val_mse: 0.1333
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1773 - mse: 0.1773 - val_loss: 0.1238 - val_mse: 0.1238
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1609 - mse: 0.1609 - val_loss: 0.1481 - val_mse: 0.1481
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1512 - mse: 0.1512 - val_loss: 0.1118 - val_mse: 0.1118
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1331 - mse: 0.1331 - val_loss: 0.1147 - val_mse: 0.1147
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1295 - 

Nicely done! After normalizing both the input and output, the model finally converged.

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`)

In [15]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 3ms/step - loss: 0.0075 - mse: 0.0075


[0.007488771807402372, 0.007488771807402372]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`)

In [16]:
# Evaluate the model on validate data
normalized_input_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 5ms/step - loss: nan - mse: nan


[nan, nan]

Since the output is normalized, the metric above is not interpretable. To remedy this:

- Generate predictions on validate data (`X_val`)
- Transform these predictions back to original scale using `ss_y`
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred`

In [17]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
mean_squared_error(y_val, y_val_pred, squared=False)

9/9 [==============================] - 0s 8ms/step


27490.923526350984

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.)

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units
  - `'relu'` activation
  - `input_shape`
  - `kernel_initializer='he_normal'`
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [18]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD',
                 loss='mse',
                 metrics=['mse'])

# Train the model
he_model.fit(X_train,
             y_train_scaled,
             batch_size=32,
             epochs=150,
             validation_data=(X_val, y_val_scaled))


Epoch 1/150
33/33 [==============================] - 1s 27ms/step - loss: 0.3471 - mse: 0.3471 - val_loss: 0.2459 - val_mse: 0.2459
Epoch 2/150
33/33 [==============================] - 0s 13ms/step - loss: 0.2460 - mse: 0.2460 - val_loss: 0.2038 - val_mse: 0.2038
Epoch 3/150
33/33 [==============================] - 0s 15ms/step - loss: 0.1910 - mse: 0.1910 - val_loss: 0.1822 - val_mse: 0.1822
Epoch 4/150
33/33 [==============================] - 0s 10ms/step - loss: 0.1595 - mse: 0.1595 - val_loss: 0.1617 - val_mse: 0.1617
Epoch 5/150
33/33 [==============================] - 0s 9ms/step - loss: 0.1479 - mse: 0.1479 - val_loss: 0.1571 - val_mse: 0.1571
Epoch 6/150
33/33 [==============================] - 0s 7ms/step - loss: 0.1359 - mse: 0.1359 - val_loss: 0.1582 - val_mse: 0.1582
Epoch 7/150
33/33 [==============================] - 0s 8ms/step - loss: 0.1216 - mse: 0.1216 - val_loss: 0.1523 - val_mse: 0.1523
Epoch 8/150
33/33 [==============================] - 0s 8ms/step - loss: 0.1113

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`)

In [19]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 4ms/step - loss: 0.0067 - mse: 0.0067


[0.006690243724733591, 0.006690243724733591]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [20]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 5ms/step - loss: 0.1323 - mse: 0.1323


[0.13231924176216125, 0.13231924176216125]

## Lecun Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units
  - `'relu'` activation
  - `input_shape`
  - `kernel_initializer='lecun_normal'`
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [22]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD',
                    loss='mse',
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train,
                y_train_scaled,
                batch_size=32,
                epochs=150,
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 10ms/step - loss: 0.4298 - mse: 0.4298 - val_loss: 0.2359 - val_mse: 0.2359
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.2330 - mse: 0.2330 - val_loss: 0.1613 - val_mse: 0.1613
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1811 - mse: 0.1811 - val_loss: 0.1382 - val_mse: 0.1382
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1646 - mse: 0.1646 - val_loss: 0.1309 - val_mse: 0.1309
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1457 - mse: 0.1457 - val_loss: 0.1261 - val_mse: 0.1261
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1358 - mse: 0.1358 - val_loss: 0.1205 - val_mse: 0.1205
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1257 - mse: 0.1257 - val_loss: 0.1279 - val_mse: 0.1279
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1128 - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`)

In [23]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 4ms/step - loss: 0.0077 - mse: 0.0077


[0.007653352338820696, 0.007653352338820696]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`)

In [24]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 7ms/step - loss: 0.1132 - mse: 0.1132


[0.1132151260972023, 0.1132151260972023]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop

Compile the `rmsprop_model` with:

- `'rmsprop'` as the optimizer
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [25]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop',
                      loss='mse',
                      metrics=['mse'])
# Code here

# Train the model
rmsprop_model.fit(X_train,
                  y_train_scaled,
                  batch_size=32,
                  epochs=150,
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 11ms/step - loss: 0.2955 - mse: 0.2955 - val_loss: 0.1061 - val_mse: 0.1061
Epoch 2/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1657 - mse: 0.1657 - val_loss: 0.0920 - val_mse: 0.0920
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1250 - mse: 0.1250 - val_loss: 0.0890 - val_mse: 0.0890
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0949 - mse: 0.0949 - val_loss: 0.0902 - val_mse: 0.0902
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0745 - mse: 0.0745 - val_loss: 0.1071 - val_mse: 0.1071
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0594 - mse: 0.0594 - val_loss: 0.0944 - val_mse: 0.0944
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0539 - mse: 0.0539 - val_loss: 0.1320 - val_mse: 0.1320
Epoch 8/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0439 - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`)

In [26]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 8ms/step - loss: 0.0059 - mse: 0.0059


[0.005869507323950529, 0.005869507323950529]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`)

In [27]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 4ms/step - loss: 0.1115 - mse: 0.1115


[0.11148101836442947, 0.11148101091384888]

## Adam

Compile the `adam_model` with:

- `'Adam'` as the optimizer
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [28]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
adam_model.compile(optimizer='Adam',
                   loss='mse',
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train,
               y_train_scaled,
               batch_size=32,
               epochs=150,
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 3s 34ms/step - loss: 0.3671 - mse: 0.3671 - val_loss: 0.1011 - val_mse: 0.1011
Epoch 2/150
33/33 [==============================] - 0s 13ms/step - loss: 0.1542 - mse: 0.1542 - val_loss: 0.0853 - val_mse: 0.0853
Epoch 3/150
33/33 [==============================] - 0s 13ms/step - loss: 0.1191 - mse: 0.1191 - val_loss: 0.0783 - val_mse: 0.0783
Epoch 4/150
33/33 [==============================] - 1s 29ms/step - loss: 0.0786 - mse: 0.0786 - val_loss: 0.0759 - val_mse: 0.0759
Epoch 5/150
33/33 [==============================] - 0s 12ms/step - loss: 0.0616 - mse: 0.0616 - val_loss: 0.0904 - val_mse: 0.0904
Epoch 6/150
33/33 [==============================] - 0s 9ms/step - loss: 0.0470 - mse: 0.0470 - val_loss: 0.0854 - val_mse: 0.0854
Epoch 7/150
33/33 [==============================] - 0s 10ms/step - loss: 0.0373 - mse: 0.0373 - val_loss: 0.0937 - val_mse: 0.0937
Epoch 8/150
33/33 [==============================] - 0s 8ms/step - loss: 0.03

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`)

In [29]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0028 - mse: 0.0028


[0.0028036644216626883, 0.0028036644216626883]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`)

In [30]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.0997 - mse: 0.0997


[0.09966405481100082, 0.09966405481100082]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [35]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 8ms/step - loss: 0.1613 - mse: 0.1613


[0.16130807995796204, 0.16130807995796204]

As earlier, this metric is hard to interpret because the output is scaled.

- Generate predictions on test data (`X_test`)
- Transform these predictions back to original scale using `ss_y`
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred`

In [36]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# RMSE of test data
mean_squared_error(y_test, y_test_pred, squared=False)


5/5 [==============================] - 0s 6ms/step


31561.637704724726

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.